Name:Suman Kumar Sahu

Dataset:
Status:

Objective:Implementing Clustering and dimensionality reduction in Spark

K-mean clustering

Elbow and silhoutte method 

Dimensionality Reduction 

Principal component Analysis

Some Hyperparameter:Number of clusters,Seeds-Initial Value, Distance Measures

Elbow method

Silhouette Coefficient; selection of outliers # read more

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession\
.builder\
.appName('Examine data about passengers on the Titanic')\
.getOrCreate()

rawData=spark.read\
.format('csv')\
.option('header','true')\
.load("/home/titan/dataset/ML_JRVI/titanic.csv")

In [3]:
rawData.toPandas().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S


In [6]:
from pyspark.sql.functions import col
dataset=rawData.select(col('Survived').cast('float'),
                      col('Pclass').cast('float'),
                      col('Sex'),
                      col('Age').cast('float'),
                      col('Fare').cast('float'),
                      col('Embarked'))

dataset.toPandas().head()


,Survived,Pclass,Sex,Age,Fare,Embarked
0,0.0,3.0,male,22.0,7.250000,S
1,1.0,1.0,female,38.0,71.283302,C
2,1.0,3.0,female,26.0,7.925000,S
3,1.0,1.0,female,35.0,53.099998,S
4,0.0,3.0,male,35.0,8.050000,S


In [7]:
dataset=dataset.replace('?',None).dropna(how='any')

In [8]:
dataset.count()

712

In [9]:
# StringIndexer:Converts categorical data to cols with numeric value

from pyspark.ml.feature import StringIndexer

dataset=StringIndexer(
inputCol='Sex',
outputCol='Gender',
handleInvalid='keep').fit(dataset).transform(dataset)

dataset=StringIndexer(
inputCol='Embarked',
outputCol='Boarded',
handleInvalid='keep').fit(dataset).transform(dataset)

dataset.toPandas().head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Gender,Boarded
0,0.0,3.0,male,22.0,7.250000,S,0.0,0.0
1,1.0,1.0,female,38.0,71.283302,C,1.0,1.0
2,1.0,3.0,female,26.0,7.925000,S,1.0,0.0
3,1.0,1.0,female,35.0,53.099998,S,1.0,0.0
4,0.0,3.0,male,35.0,8.050000,S,0.0,0.0


In [10]:
dataset=dataset.drop('Sex','Embarked')

In [11]:
dataset.toPandas().head()

,Survived,Pclass,Age,Fare,Gender,Boarded
0,0.0,3.0,22.0,7.250000,0.0,0.0
1,1.0,1.0,38.0,71.283302,1.0,1.0
2,1.0,3.0,26.0,7.925000,1.0,0.0
3,1.0,1.0,35.0,53.099998,1.0,0.0
4,0.0,3.0,35.0,8.050000,0.0,0.0


In [12]:
requiredFeatures=['Survived',
                  'Pclass',
                  'Age',
                  'Fare',
                  'Gender',
                  'Boarded']

In [20]:
from pyspark.ml.feature import VectorAssembler


assembler=VectorAssembler(inputCols=requiredFeatures,
                          outputCol='features')

In [18]:
transformed_data=assembler.transform(dataset)

Exception ignored in: <function JavaWrapper.__del__ at 0x7fce05dcfc20>
Traceback (most recent call last):
  File "/home/titan/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7fce05dcfc20>
Traceback (most recent call last):
  File "/home/titan/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [19]:
help('VectorAssembler')

No Python documentation found for 'VectorAssembler'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

